# Model Testing


In [1]:
from scripts.preprocessing.data_loader import get_train_test_splits, get_combined_dataset
from scripts.preprocessing.preprocessor import apply_minmax_scaling
from scripts.models.model_list import load_models
from scripts.models.model_testing import fit_and_evaluate_model
from IPython.display import display

X_train, X_test, y_train, y_test = get_train_test_splits(test_size=0.2)
X_train, y_train, X_train_scaler, y_train_scaler = apply_minmax_scaling(
    X_train, y_train
)
X_test, y_test, X_test_scaler, y_test_scaler = apply_minmax_scaling(X_test, y_test)
df = get_combined_dataset(X_train, y_train, X_test, y_test)
display(df)

TEST_SET, ALL_SINGLE_TARGET_MODELS, REFINED_SINGLE_TARGET_MODELS, ALL_MULTI_TARGET_MODELS, REFINED_MULTI_TARGET_MODELS = load_models()
print(f"Number of single-target models: {len(ALL_SINGLE_TARGET_MODELS)}")
print(f"Number of multi-target models: {len(ALL_MULTI_TARGET_MODELS)}")

Train years: 2015 to 2083
Test years: 2084 to 2100


,x,y,bedrock_elevation,precipitation,air_temperature,ocean_temperature,year,distance_to_pole,bedrock_below_sea_level,set,ice_thickness,ice_velocity,ice_mask
0,0.0,1.0,0.118762,0.180874,0.795808,0.268660,2015,0.0,1.0,train,0.0,0.0,1.0
1,0.0,1.0,0.163009,0.175715,0.793057,0.258708,2015,0.0,1.0,train,0.0,0.0,1.0
2,0.0,1.0,0.042803,0.173047,0.791427,0.251029,2015,0.0,1.0,train,0.0,0.0,1.0
3,0.0,1.0,0.135461,0.178002,0.790789,0.245280,2015,0.0,1.0,train,0.0,0.0,1.0
4,0.0,1.0,0.087251,0.190951,0.790841,0.240754,2015,0.0,1.0,train,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
194097,0.0,0.0,0.347665,0.592466,0.831212,0.272332,2100,0.0,1.0,test,0.0,0.0,1.0
194098,0.0,0.0,0.326669,0.557774,0.838175,0.285703,2100,0.0,1.0,test,0.0,0.0,1.0
194099,0.0,0.0,0.365826,0.535849,0.843655,0.296153,2100,0.0,1.0,test,0.0,0.0,1.0
194100,0.0,0.0,0.416587,0.533697,0.851775,0.315210,2100,0.0,1.0,test,0.0,0.0,1.0


Number of single-target models: 25
Number of multi-target models: 10


## Single Target Models


### Broad Testing


In [5]:
single_target_results = {}
for name, model in ALL_SINGLE_TARGET_MODELS:
    print(f"Evaluating {name} for single-target regression:")
    results = fit_and_evaluate_model(
        model, X_train, y_train, X_test, y_test, multi=False, cv=5
    )
    single_target_results[name] = results

for name, results in single_target_results.items():
    print(f"\n{name}:")
    display(results)

Evaluating Linear Regression for single-target regression:
Evaluating Ridge for single-target regression:
Evaluating Lasso for single-target regression:
Evaluating ElasticNet for single-target regression:
Evaluating Lars for single-target regression:
Evaluating LassoLars for single-target regression:
Evaluating OrthogonalMatchingPursuit for single-target regression:
Evaluating BayesianRidge for single-target regression:
Evaluating ARDRegression for single-target regression:
Evaluating SGDRegressor for single-target regression:
Evaluating PassiveAggressiveRegressor for single-target regression:
Evaluating HuberRegressor for single-target regression:
Evaluating RANSACRegressor for single-target regression:


C:\Users\samma\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\samma\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\samma\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\samma\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\samma\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:121

Evaluating TheilSenRegressor for single-target regression:
Evaluating LinearSVR for single-target regression:


C:\Users\samma\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\samma\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\samma\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\samma\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\samma\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\samma\AppData\Roaming\Python\Python3

Evaluating KNeighborsRegressor for single-target regression:
Evaluating DecisionTreeRegressor for single-target regression:
Evaluating RandomForestRegressor for single-target regression:
Evaluating ExtraTreesRegressor for single-target regression:
Evaluating GradientBoostingRegressor for single-target regression:
Evaluating AdaBoostRegressor for single-target regression:
Evaluating MLPRegressor for single-target regression:
Evaluating PLSRegression for single-target regression:
Evaluating XGBoost for single-target regression:
Evaluating LightGBM for single-target regression:
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000573 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1083
[LightGBM] [Info] Number of data points in the train set: 124586, number of used features: 8
[LightGBM] [Info] Start training from score 0.168956
[LightGBM] [

...

,Target,MSE,RMSE,MAE,R2
0,ice_thickness,0.013589,0.116574,0.080526,0.789626
1,ice_velocity,0.000158,0.012580,0.003260,0.023283
2,ice_mask,0.050089,0.223805,0.161985,0.781333



XGBoost:


,Target,MSE,RMSE,MAE,R2
0,ice_thickness,0.011933,0.109238,0.091619,0.815272
1,ice_velocity,0.000169,0.012999,0.002879,-0.042789
2,ice_mask,0.041059,0.202630,0.188063,0.820753



LightGBM:


,Target,MSE,RMSE,MAE,R2
0,ice_thickness,0.012103,0.110014,0.092334,0.812638
1,ice_velocity,0.000119,0.010890,0.002545,0.268137
2,ice_mask,0.040738,0.201835,0.187511,0.822156


### Refined Testing


In [2]:
TEST_SET, ALL_SINGLE_TARGET_MODELS, REFINED_SINGLE_TARGET_MODELS, ALL_MULTI_TARGET_MODELS, REFINED_MULTI_TARGET_MODELS = load_models(N_ESTIMATORS=1000, MIN_SAMPLES_SPLIT=5)

refined_single_target_results = {}
for name, model in REFINED_SINGLE_TARGET_MODELS:
    print(f"Evaluating {name} for refined single-target regression:")
    results = fit_and_evaluate_model(
        model, X_train, y_train, X_test, y_test, multi=False, cv=5
    )
    refined_single_target_results[name] = results

for name, results in refined_single_target_results.items():
    print(f"\n{name}:")
    display(results)

Evaluating RandomForestRegressor for refined single-target regression:


### Model Interpretation


## Multi-Target Models


In [4]:
single_target_results = {}
for name, model in ALL_MULTI_TARGET_MODELS:
    print(f"Evaluating {name} for multi-target regression:")
    results = fit_and_evaluate_model(
        model, X_train, y_train, X_test, y_test, multi=True, cv=5
    )
    single_target_results[name] = results

for name, results in single_target_results.items():
    print(f"\n{name}:")
    display(results)

Evaluating Multi-target Linear Regression for multi-target regression:
Evaluating Multi-target Ridge for multi-target regression:
Evaluating Multi-target Lasso for multi-target regression:
Evaluating Multi-target ElasticNet for multi-target regression:
Evaluating Multi-target RandomForest for multi-target regression:
Evaluating Multi-target ExtraTrees for multi-target regression:
Evaluating Multi-target GradientBoosting for multi-target regression:
Evaluating Multi-target MLPRegressor for multi-target regression:
Evaluating Multi-target XGBoost for multi-target regression:
Evaluating Multi-target LightGBM for multi-target regression:
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000502 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1083
[LightGBM] [Info] Number of data points in the train set: 124586, number of used features: 8
[Ligh

,Target,MSE,RMSE,MAE,R2
0,ice_thickness,0.007701,0.087756,0.060153,0.880783
1,ice_velocity,0.000157,0.012547,0.003300,0.028451
2,ice_mask,0.042598,0.206393,0.150208,0.814034



Multi-target Ridge:


,Target,MSE,RMSE,MAE,R2
0,ice_thickness,0.007701,0.087755,0.060152,0.880783
1,ice_velocity,0.000157,0.012547,0.003300,0.028451
2,ice_mask,0.042598,0.206393,0.150207,0.814034



Multi-target Lasso:


...

,Target,MSE,RMSE,MAE,R2
0,ice_thickness,0.289873,0.538398,0.410515,-3.487417
1,ice_velocity,0.123782,0.351827,0.308238,-762.928056
2,ice_mask,0.110393,0.332254,0.271859,0.518070



Multi-target XGBoost:


,Target,MSE,RMSE,MAE,R2
0,ice_thickness,0.011933,0.109238,0.091619,0.815272
1,ice_velocity,0.000169,0.012999,0.002879,-0.042789
2,ice_mask,0.041059,0.202630,0.188063,0.820753



Multi-target LightGBM:


,Target,MSE,RMSE,MAE,R2
0,ice_thickness,0.012103,0.110014,0.092334,0.812638
1,ice_velocity,0.000119,0.010890,0.002545,0.268137
2,ice_mask,0.040738,0.201835,0.187511,0.822156


## Sequential Models
